# BOM Comparer
### Take an old BOM sheet and a new BOM sheet and compares what is new or missing.

In [1]:
# xlsx comparer
# purpose: find difference between two xlsx files and indicate them in a new file.
# library to use: pandas, xlsxwriter
# Update 2.1: 
# + Added variable assignments to input values in order to increase ease of use.
from sys import exit
import pandas as pd
import xlsxwriter
import PySimpleGUI as sg
pd.set_option('display.max_columns', 10)

### IMPORTANT SECTION: Edit Inputs Here ###
# the file path to the old file.

old_file = "Luke_P1_DFLT_BOM_v1.xlsx"

# the file path to the new file.
new_file = "Kevin_P1_RF_BOM_v1.xlsx"

# unique item identifier. input a column name that identifies unique items, such as LibRef.
uid = "LibRef"

### End of Inputs Section ###

# Helper method to edit cells based on met conditions. 
# Input: series containing the values of old and new 
# Output: string to replace the cell.
def report_diff(x):
    if x[0] == x[1]:
        return x[0]
    elif x.isnull()[0]:
        return 'ADDED ---> %s' % x[1]
    elif x.isnull()[1]:
        return '%s ---> REMOVED' % x[0]
    else:
        return '{} ---> {}'.format(*x)

def file_compare(old_file, new_file):
    # *** Read the files, old vs new ***
    old = pd.read_excel(old_file, sheet_name=None)
    new = pd.read_excel(new_file, sheet_name=None)
    finalWriter = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
    # Text wrap formatting, currently obsolete and incompatible with highlighting.
    workbook = finalWriter.book
    cell_format = workbook.add_format()
    cell_format.set_text_wrap(True)

    # *** for loop to iterate through multiple pages in the excel file.
    # Assumes old and new have the same page length and contents for each page ***
    for i in range(len(list(old))):
        old_df = (list(old.values())[i])
        new_df = (list(new.values())[i])
        # Creating a new DataFrame df using
        # 1. merge and the flags required for a comparison table (indicator & how).
        # 2. a lambda function that returns only values NOT present in both old and new (!='both')
        df = (old_df.merge(new_df, indicator=True, how='outer')).loc[lambda v: v['_merge'] != 'both']
        df.reset_index(inplace=True, drop=True)
        df.rename(columns={'_merge': 'version'}, inplace=True)
        df['version'] = df['version'].replace(['left_only'], 'old')
        df['version'] = df['version'].replace(['right_only'], 'new')

        # This code was derived from: https://pbpython.com/excel-diff-pandas-update.html
        # Make new DataFrames based on the categories of 'old' and 'new' from the base DataFrame.
        old_sep = df[(df['version'] == 'old')]
        new_sep = df[(df['version'] == 'new')]

        old_sep.reset_index(inplace=True, drop=True)
        new_sep.reset_index(inplace=True, drop=True)
    
        old_sep = old_sep.drop(['version'], axis=1)
        new_sep = new_sep.drop(['version'], axis=1)
    
        # Set uid as the index for proper indexing of rows. 
        # Any column with static, unique data can work for uid.
        if uid in df:
            old_sep.set_index(old_sep[uid], inplace=True)
            new_sep.set_index(new_sep[uid], inplace=True)

        # Make a new DataFrame by combining old_sep and new_sep. This makes side-by-side tables
        # of the old and new sheets.
        df = pd.concat([old_sep, new_sep], axis='columns', keys=['old', 'new'], join='outer')

        # Swap the levels between the descriptor categories (Manufacturer #, Designator, LibRef, etc. ) 
        # and the version categories (old & new). The result is a df that displays old and new versions of
        # each descriptor in side-by-side cells rather than side-by-side tables.
        df = df.swaplevel(axis='columns')[new_sep.columns[0:]]
        # Makes a df that groups cells by descriptor category(level=0) and column(axis=1), turning each group into a 2-item series 
        # containing the old and new items. 
        # Performs report_diff to these series and applies the result to the cell within df.groupby.
        df_changed = df.groupby(level=0, axis=1).apply(lambda frame: frame.apply(report_diff, axis=1))
        df_changed = df_changed.reset_index(drop=True)
        df = df_changed
    
        # CODE FOR HIGHLIGHTING THE SHEET, gold = modified item, red = removed item, green = new item.
        # TODO: Highlight the sheet in way that does not interfere with text wrap formatting.
        if uid in df:
            df_styled = df.style\
                        .applymap(lambda x: 'background-color: gold' if '--->' in str(x) else '')\
                        .apply(lambda x: ['background-color: salmon' if 'REMOVED' in x else '' for x in df[uid]], axis=0)\
                        .apply(lambda x: ['background-color: mediumseagreen' if 'ADDED' in x else '' for x in df[uid]], axis=0)
        else:
            df_styled = df.style\
                        .applymap(lambda x: 'background-color: gold' if '--->' in str(x) else '')
    
        #display(df_styled)
        
        # Put 'df_styled' instead of 'df' to get a highlighted sheet.
        df_styled.to_excel(finalWriter, 'Sheet%d' % (i + 1))  # send df to writer
           
        worksheet = finalWriter.sheets['Sheet%d' % (i + 1)]  # pull worksheet object
        worksheet.set_column('B:Z', 15, cell_format)

    finalWriter.save()




In [2]:
'''
    Simple "diff" in PySimpleGUI
'''

sg.theme('Dark Blue 3')

def GetFilesToCompare():
    form_rows = [[sg.Text('Enter 2 files to compare')],
                 [sg.Text('File 1', size=(15, 1)),
                    sg.InputText(key='-file1-'), sg.FileBrowse()],
                 [sg.Text('File 2', size=(15, 1)), sg.InputText(key='-file2-'),
                  sg.FileBrowse(target='-file2-')],
                 [sg.Submit(), sg.Cancel()]]

    window = sg.Window('File Compare', form_rows)
    event, values = window.read()
    window.close()
    return event, values


def main():

    button, values = GetFilesToCompare()
    f1, f2 = values['-file1-'], values['-file2-']

    if any((button != 'Submit', f1 == '', f2 == '')):
        sg.popup('Operation cancelled')
        exit()
        
    # --- This portion of the code is not GUI related ---
    file_compare(f1, f2)
    sg.popup('File Comparison Complete')
    exit()
    

if __name__ == '__main__':
    main()

objc[8234]: Class FIFinderSyncExtensionHost is implemented in both /System/Library/PrivateFrameworks/FinderKit.framework/FinderKit (0x12b24b3d8) and /System/Library/PrivateFrameworks/FileProvider.framework/OverrideBundles/FinderSyncCollaborationFileProviderOverride.bundle/Contents/MacOS/FinderSyncCollaborationFileProviderOverride (0x13353ff50). One of the two will be used. Which one is undefined.
objc[8234]: Class SLWebTokenHandlerController is implemented in both /System/Library/PrivateFrameworks/SocialServices.framework/Versions/A/SocialServices (0x7fffa9a63268) and /System/Library/Frameworks/Social.framework/Versions/A/Social (0x132275820). One of the two will be used. Which one is undefined.


SystemExit: 

/Users/kevinnguyen/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
